In [1]:
import os
import pytesseract
from docx2pdf import convert
import subprocess
import time
import win32com.client  # Убедитесь, что этот импорт присутствует
import fitz  # PyMuPDF
from PIL import Image, ImageDraw, ImageFont
import tkinter as tk
from tkinter import messagebox

In [2]:
#  ----   Основные переменные для пользоваетля

input_dir = './input' # в этой папке лежат .docx-файлы лекций
pdf_dir = './pdf' # в эту папку скрипт сконвертирует .docx-файлы из input_dir. (это необходимо, т.к. скрипт видит страницы именно в pdf формате (а не в docx))

 # в этой папке скрипт создаст (для каждой лекции из input) свою подпапку
output_dir = './output'
# в папке output, в подпапке каждой лекции лежат две папки: to_combine и to_print

# В этой папке лежат сделанные из pdf картинки страниц текста
# эта папка лежит в output_dir.
to_combine_dir = './to_combine' 

# В этой папке лежат объединенные специфическим образом для печати на А4 картинки из папки to_combine
# Это и есть готовые конспекты к печати
to_print = './to_print'

# Загружаем шрифт нашего почерка
font = ImageFont.truetype("./fonts/MyFont.otf", size=95)

note_type = 'cells' #клеточки. note_type='lines' - лист в линейку

# Задаем вертикальный шаг между строками
line_spacing = 110  # 55 - писать на каждой строке, 110 - пропускать одну строку между двумя строками текста

#Адрес файла запуска Microsoft Word
word_path = r"C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE" 


In [3]:
#очищаем папку output от картинок и pdf-файлов из предыдущих итераций
def delete_files_in_folder(folder):
    if os.path.exists(folder):  # Проверяем, что папка существует
        for root, dirs, files in os.walk(folder):
            for file in files:
                file_path = os.path.join(root, file)
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.pdf')):
                    print(f'Удаление файла: {file_path}')
                    os.remove(file_path)  # Удаляем файл
    else:
        print(f"Папка {folder} не существует.")

# пользователю задаётся вопрос: очистить папки: input_dir, pdf_dir, output_dir, to_combine_dir
# пользователь может поставить галочки в чекбоксы напротив нужных ему папок
# после чего идет цикл по выбранным папкам, и функция delete_files_in_folder очищает содержимое

In [4]:
# Функция для обработки выбранных папок
def delete_selected_folders():
    selected_folders = []
    
        # Функция для поиска и удаления файлов во всех папках to_print и to_combine
    def delete_in_all_subfolders(base_folder, target_folder_name):
        for root, dirs, files in os.walk(base_folder):
            if target_folder_name in dirs:
                target_folder_path = os.path.join(root, target_folder_name)
                print(f"Очистка папки: {target_folder_path}")
                delete_files_in_folder(target_folder_path)
    
    # Проверяем, какие чекбоксы выбраны
    if var_pdf_dir.get():
        selected_folders.append(pdf_dir)
    if var_to_print.get():
        # Удаление файлов во всех папках to_print в папке output
        delete_in_all_subfolders('./output', 'to_print')
    if var_to_combine_dir.get():
        # Удаление файлов во всех папках to_combine в папке output
        delete_in_all_subfolders('./output', 'to_combine')

    if selected_folders:
        # Подтверждение действия для папок input и pdf
        result = messagebox.askquestion("Подтверждение", "Вы уверены, что хотите очистить выбранные папки?")
        if result == 'yes':
            for folder in selected_folders:
                print(f"Очистка папки: {folder}")
                delete_files_in_folder(folder)  # Очищаем выбранную папку
            messagebox.showinfo("Готово", "Выбранные папки успешно очищены.")
        else:
            messagebox.showinfo("Отмена", "Очистка папок отменена.")

In [5]:
#В папке output могут быть сколько угодно разных папок, внутри которых могут быть папки to_print и to_combine_dir
#Если пользователь выберет чекбокс удалить to_print/to_combine_dir, то удалится должны все файлы во всех папках to_print/to_combine_dir

# Папки для очистки
input_dir = './input'
pdf_dir = './pdf'
to_print_dir = './to_print'
to_combine_dir = './to_combine'


# Создаем окно
root = tk.Tk()
root.title("Очистка папок")

# Создаем переменные для чекбоксов
var_pdf_dir = tk.BooleanVar()
var_to_combine_dir = tk.BooleanVar()
var_to_print = tk.BooleanVar()

# Создаем чекбоксы
tk.Checkbutton(root, text="Очистить папку pdf", variable=var_pdf_dir).pack(anchor=tk.W)
tk.Checkbutton(root, text="Очистить папку output/to_print", variable=var_to_print).pack(anchor=tk.W)
tk.Checkbutton(root, text="Очистить папку output/to_combine", variable=var_to_combine_dir).pack(anchor=tk.W)

# Кнопка для выполнения очистки
tk.Button(root, text="Очистить выбранные папки", command=delete_selected_folders).pack(pady=10)

# Запуск окна
root.mainloop()


In [6]:
def create_dir_if_it_doesnt_exist(root_dir, dir):
    path = os.path.join(root_dir, dir,)
        
    if not os.path.exists(path):
        os.makedirs(path)
        
create_dir_if_it_doesnt_exist('./', input_dir)
create_dir_if_it_doesnt_exist('./', pdf_dir)
create_dir_if_it_doesnt_exist('./', output_dir)


In [7]:
# Убедитесь, что путь к изображению корректный и изображение загружено
img_template_path=''

if note_type == 'cells':
    img_template_path = './img/cells_half.png'
else:
    img_template_path = './img/lines_half.png'


if not os.path.exists(img_template_path):
    raise FileNotFoundError(f"Изображение не найдено по пути: {img_template_path}")

# Открываем изображение с клетками
img_template = Image.open(img_template_path)

In [8]:
def run_ms_word(word_path):
    # Запускаем Microsoft Word
    try:
        is_ms_word_closed = subprocess.Popen([word_path])
        print("Microsoft Word запущен.")
    except Exception as e:
        print(f"Не удалось запустить Microsoft Word: {e}")
        

In [9]:
def is_ms_word_running(process_name="WINWORD.EXE"):
    """Проверяет, работает ли Microsoft Word и полностью ли он загружен."""
    try:
        # Получаем список процессов и проверяем наличие Microsoft Word
        output = subprocess.check_output(["tasklist"], text=True)
        
        if process_name in output:
            print("MS Word работает.")
            
            # Дожидаемся полной загрузки Word
            while True:
                try:
                    # Проверяем, доступен ли объект Word
                    word_app = win32com.client.GetActiveObject("Word.Application")
                    if word_app is not None:
                        print("MS Word полностью загружен и готов к работе.")
                        return True
                except Exception:
                    # Если объект недоступен, продолжаем ожидание
                    print("Ожидание полной загрузки MS Word...")
                    time.sleep(1)  # Ждем 1 секунду перед следующей проверкой
            
        else:
            print("MS Word не запущен.")
            return False
    except Exception as e:
        print(f"Ошибка при проверке процессов: {e}")
        return False
    

In [10]:
for filename in os.listdir(input_dir):
    if ".docx" in filename.lower() and "$" not in filename:
        print(filename)
        
        filename_no_ext = filename[:-5]
        temp_pdf_path = (f"{pdf_dir}/{filename_no_ext}.pdf")
        

        # Если Word запущен через subroccess:
        if is_ms_word_running():
             # Microsoft Word не успевает запуститься до запуска этой строки. Как исправить?
            convert( (f"{input_dir}/{filename}"), temp_pdf_path)
        
        
        #Иначе запускаем его и 
        else:
            run_ms_word(word_path)
            # Как здесь дождаться когда Microsoft запустится? После этого нужно выполнить этот код:
            

            # Ожидаем, пока Microsoft Word не запустится
            while not is_ms_word_running():
                print("Ожидание запуска Microsoft Word...")
                time.sleep(2)  # Ждем 1 секунду перед следующей проверкой

            # После того как Microsoft Word запущен, выполняем конвертацию
            try:
                convert(os.path.join(input_dir, filename), temp_pdf_path)
                print(f'Конвертировано: {input_dir}/{filename} в {temp_pdf_path}')
            except Exception as e:
                print(f'Ошибка при конвертации: {e}')

  
        print(f"{input_dir}/{filename}")
        

текст.docx
MS Word работает.
MS Word полностью загружен и готов к работе.


  0%|          | 0/1 [00:00<?, ?it/s]

./input/текст.docx


In [11]:
for input_filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, input_filename)
        
    if os.path.isfile(input_path) and 'docx' in input_filename.lower() and '$' not in input_filename:
        print(input_filename)
        create_dir_if_it_doesnt_exist(output_dir, input_filename[:-5])

текст.docx


In [12]:
for input_filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, input_filename)
    

    
    if os.path.isfile(input_path) and 'docx' in input_filename.lower() and '$' not in input_filename:
        #print(input_filename)
        create_dir_if_it_doesnt_exist(output_dir, input_filename[:-5])

        # Открываем PDF-документ
        pdf_path = os.path.join(pdf_dir, f'{input_filename[:-5]}.pdf')
        
        create_dir_if_it_doesnt_exist(f'{output_dir}/{input_filename[:-5]}', to_combine_dir)

        doc = fitz.open(pdf_path)

        # Перебираем страницы PDF
        for page_num in range(len(doc)):
            # Получаем страницу
            page = doc.load_page(page_num)
            
            # Извлекаем текст с форматированием
            text = page.get_text("text")  # Извлекаем текст с форматированием
            
            # Разбиваем текст на отдельные строки
            text_lines = text.splitlines()

            print(text)
            print('_____')

            # Создаем копию шаблона для каждой страницы
            img = img_template.copy()

            

            draw = ImageDraw.Draw(img)

            # Переносим строки текста на соответствующие линии
            for i, line in enumerate(text_lines):
                # Координата Y для нижней части строки

                line__edited = line
                
                # Убираем капслок
                # line__edited = line.lower()

                # Убираем буквы "бвдзруфцщ"
                # line__edited = ''.join([char for char in line__edited if char not in chars_to_remove])
                
                # Берем высоту буквы по англ. символу "a"
                char_a = 'a' 
                line_height = font.getsize(char_a)[1]  # Получаем высоту строки
                
                y_position = (i + 2) * line_spacing - line_height  # Выравниваем низ текста по линии

                # Рисуем текст с заданной позицией по оси Y
                draw.text((4 * 55, y_position), line, fill="black", font=font)

            # Если страница первая или четвертая, поворачиваем изображение на 180 градусов
            #if page_num == 0 or page_num == 3:
            #  img = img.rotate(180)

            # Сохраняем результат для текущей страницы
            img.save(f'{output_dir}/{input_filename[:-5]}/{to_combine_dir}/{page_num + 1}.png')  # Нумерация страниц начинается с 1

        # Закрываем PDF-документ
        doc.close()

1. Настройки Microsoft Word: 
     Шрифт: MyFont, Размер: 24 
(шрифт нужно установить на ПК) 
 
      Поля:  
     Верхнее: 3,5см,  
     Левое: 0,3см,  
     Нижнее: 2см,  
     Правое: 0,5см,  
     Переплет: 2см,  
     Положение переплета: слева  
____________________________ 
2. Переносы текста в MS Word должны быть 
сделаны через shift+enter, чтобы перенос 
строк был размером как строка текста. 
3. Стили текста: жирный, курсив, подчеркнутый 
– не поддерживаются 

_____
4. Заголовок должен быть сделан 
пробелами/табами слева 
 
             3. Заголовок 1 
4 Текст: Lorem ipsum dolor sit amet, 
consetetur sadipscing elitr, sed diam 
nonumy eirmod tempor invidunt ut labore 
et dolore magna aliquyam erat, sed 
diam voluptua. At vero eos et accusam 
et justo duo dolores et ea rebum. Stet 
clita kasd gubergren, no sea takimata 
sanctus est Lorem ipsum dolor sit amet. 
Lorem ipsum dolor sit amet, consetetur 
sadipscing elitr, sed diam nonumy eirmod 
tempor invidunt ut labore et dolore 


C:\Users\Personal Computer\AppData\Local\Temp\ipykernel_19112\2703269169.py:52: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  line_height = font.getsize(char_a)[1]  # Получаем высоту строки


Текст: Lorem ipsum dolor sit amet, 
consetetur. Картинка: 
Обтекание текстом 
(картинка справа)  
 
Формула: 𝑓1(𝑥) =
1
5 𝑥4 −2𝑥3 +
3
2 ; 𝑓2(𝑥) =
4
𝑥−1 +
𝑥2+1
𝑥+5  
(Преобразовано в Ворде в Office Math ML 
(по двойному щелчку по формуле)) 
𝑓1(𝑥) = 1
5 𝑥4 −2𝑥3 + 3
2 ; 𝑓2(𝑥)
=
4
𝑥−1 + 𝑥2 + 1
𝑥+ 5  
 
 

_____
Формула: 
5
1
1
4
)
(
;
2
3
2
5
1
)
(
2
2
3
4
1
+
+
+
−
=
+
−
=
x
x
x
x
f
x
x
x
f
 
(Microsoft equation, т.е. не 
преобразовано) 
____________________________ 
 
 

_____
График: 
   у 
                f2(x) = 
x
1 
 
  
 
 
 
 
f1(x) = 
x
lg  
 
  
 
 
 
        х 
 
  
2           3 
3,2
,
48
,0
3
lg
,3,0
2
lg




. 
 
 
 
 

_____
6-я страница. Разрыв снизу сделано 
многочисленным нажатиями shift+enter 
(enter ,без shift на последней строке) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

_____
7-я страница 

_____
